The pose can be represented as a homogeneous transformation matrix:

$X=\begin{bmatrix} R & t\\ 0 & 1 \end{bmatrix} = \begin{bmatrix} cos(\theta) & -sin(\theta) & x\\ sin(\theta) & cos(\theta) & y \\ 0 & 0 & 1 \end{bmatrix}$

Or as a 3 compoent vector:

$x=[x, y, \theta]$

We can define the functions to convert pose vector to the transformation matrix and vice versa:

In [2]:
import numpy as np
from numpy.linalg import inv

def t2v(tr):
    # homogeneous transformation to vector
    v = np.zeros((3,1))
    v[:2, 0] = tr[:2,2]
    v[2] = np.arctan2(tr[1,0], tr[0,0])
    return v

def v2t(v):
    # vector to homogeneous transformation
    c = np.cos(v[2])
    s = np.sin(v[2])
    tr = np.array([[c, -s, v[0]],
                   [s,  c, v[1]],
                   [0,  0,  1]])
    return tr

The pose graph edge error function can be written as the following form:

$e_{ij}(x_i,x_j)=t2v(Z_{ij}^{-1}(X_{i}^{-1}X_{j}))$

Where $Z_{ij} = \begin{bmatrix} R'& t'\\ 0 & 1 \end{bmatrix}$ is the measurement associated with pose graph edge. 

Notice that the position and orientation difference between the pose $x_i$  and the pose $x_j$ is written in a form of the transformation matrix. We can show that the multiplication of the two transformation matrices can express their difference.  

Let $X_{wi}$ represents the pose $i$ in the world coordinate system $w$, and also represents the conversion matrix that converts a point in the coordinate system $i$ to the world coordinate system $w$. Similarly $X_{wj}$ is the pose $j$ in the world coordinate system $w$, and represents the conversion matrix that converts a point in the coordinate system $j$ to the world coordinate system $w$.

So the following umtiplication $X_{wi}^{-1}\cdot X_{wj} = X_{iw}\cdot X_{wj}=X_{ij}$ represents the conversion matrix from the coordinate system $j$ to the coordinate system $i$.



In [196]:
# Pose i
v_i = np.array([0, 0, 0])

# Pose j
v_j = np.array([0.7, 0, np.pi / 2])

# Edge measurement
v_z = np.array([1, 0, np.pi / 3])

def calculate_error(x_i, x_j, z):
  # Create transformation matrices from vectors
  t_i = v2t(x_i)
  t_j = v2t(x_j)
  t_z = v2t(z)

  # Get rotation matrices
  r_i = t_i[:2,:2]
  r_z = t_z[:2,:2]

  # Calculate error vector
  e = t2v(inv(t_z) @ (inv(t_i) @ t_j))
  return e

e = calculate_error(v_i, v_j, v_z)
display(e)

array([[-0.15      ],
       [ 0.25980762],
       [ 0.52359878]])

In [195]:
# Pose i
v_i = np.array([0, 0, 0])

# Pose j
v_j = np.array([1, 0, np.pi / 3])

# Edge measurement
v_z = np.array([1, 0, np.pi / 3])

e = calculate_error(v_i, v_j, v_z)
display(e)

array([[0.00000000e+00],
       [0.00000000e+00],
       [4.06369831e-17]])

Error should take a value of zero if:

$Z_{ij} = (X_{i}^{-1}X_{j})$

In [194]:
# Pose i
v_i = np.array([0, 0, 0])

# Pose j
v_j = np.array([1, 0, np.pi / 3])

# Edge measurement
v_z = np.array([1, 0, np.pi / 3])

e = calculate_error(v_i, v_j, v_z)
display(e)

array([[0.00000000e+00],
       [0.00000000e+00],
       [4.06369831e-17]])

The next important part is calculating Jacobian of the error function. Lets make some preparations.

According to the previous explanation we have:

$X_{i}^{-1}X_{j} = \begin{bmatrix} R & t\\ 0 & 1 \end{bmatrix}$

Inversion of a block transformation matrix can be expresed with the following formula:

$\begin{bmatrix} R & t\\ 0 & 1 \end{bmatrix}^{-1} = \begin{bmatrix} \begin{bmatrix} I & t\\ 0 & 1 \end{bmatrix} & \begin{bmatrix} R & 0 \\ 0 & 1 \end{bmatrix} \end{bmatrix}^{-1} = \begin{bmatrix} \begin{bmatrix} R & 0 \\ 0 & 1 \end{bmatrix}^ {-1} & \begin{bmatrix} I & t\\ 0 & 1 \end{bmatrix}^{-1} \end{bmatrix}=\begin{bmatrix} \begin{bmatrix} R^{T} & 0\\ 0 & 1 \end{bmatrix}&\begin{bmatrix} I & -t\\ 0 & 1 \end{bmatrix} \end{bmatrix}= \begin{bmatrix} R^{T} & -R^{T}t\\ 0 & 1 \end{bmatrix}$

So we can rewrite the error function in the following form:

$Z_{ij}^{-1}(X_{i}^{-1}X_{j}) = \begin{bmatrix} R' & t'\\ 0 & 1 \end{bmatrix}^{-1} \begin{bmatrix} R & t\\ 0 & 1 \end{bmatrix}=\begin{bmatrix} R'^{T} & -R't'\\ 0 & 1 \end{bmatrix}\begin{bmatrix} R & t\\ 0 & 1 \end{bmatrix}=\begin{bmatrix} R'^{T}R & Rt-R't'\\ 0 & 1 \end{bmatrix}=\begin{bmatrix}R'^{T}R & R'^{T}(t-t')\\0 & 1\end{bmatrix}$

And the same function in the vectorized form:

$e_{ij}(x_i,x_j)=Z_{ij}^{-1}(\begin{bmatrix}R_i^{T}(t_j-t_i)\\ \theta_j - \theta_i \end{bmatrix})$

$\Delta t_{ij} = R_{z}^{T}\begin{bmatrix} R_{i}^{T}(\begin{bmatrix} x_j \\ y_j\end{bmatrix} - \begin{bmatrix} x_i \\ y_i\end{bmatrix}) - \begin{bmatrix} x_z \\ y_z\end{bmatrix})\end{bmatrix}$

$\Delta \theta_{ij} = (\theta_j - \theta_i) - \theta_z$


The form of the $X$ vector for the optimization problem - is one long vector stacked from all pose vectors:

$x =[(x_1, y_1, \theta_1), (x_2, y_2, \theta_2), ...., (x_n, y_n, \theta_n)]$

Our error term doesn't depend on all state variables but on only on 3 values blocks related to the $i$-th and $j$-th poses.

And this fact is also visible in the form of Jacobian:

$J_{ij} = \begin{bmatrix} 0 & ... & 0 & A_{ij} &  0 & ... & 0 & B_{ij} & 0 & ... & 0 \end{bmatrix}$

$A_{ij} = \frac{\partial e_{ij}(x_i)}{\partial x_i}$

$B_{ij} = \frac{\partial e_{ij}(x_j)}{\partial x_j}$

So for $A_{ij}$ and $B_{ij}$ we have the following formulas:

$A_{ij} = \begin{bmatrix} -R_{z}^T R_{i}^T &  R_{z}^T \frac{\partial R_{i}^T}{\partial \theta_i} (t_j - t_i) \\ 0 & -1 \end{bmatrix}$

where:

1. the 3x2 block $\begin{bmatrix}-R_{z}^T R_{i}^T \\ 0 \end{bmatrix}$ is the partial derrivative of $\Delta t_{ij}$ in the top and  $\Delta \theta_{ij}$ in the bottom w.r.t $\left[x_i y_i\right]$

2. the 3x1 block $\begin{bmatrix} R_{z}^T \frac{\partial R_{i}^T}{\partial \theta_i} (t_j - t_i) \\ -1 \end{bmatrix}$ is the partial derrivative of $\Delta t_{ij}$ in the top and  $\Delta \theta_{ij}$ in the bottom w.r.t $\theta_i$

3. $\frac{\partial R_{i}^T}{\partial \theta_i} = \begin{bmatrix} -sin(\theta_i) & cos(\theta_i) \\  -cos(\theta_i) & -sin(\theta_i) \end{bmatrix}$

In the same manner we take partial derrivatives w.r.t $\left[x_j y_j\right]$ and $\theta_j$:


$B_{ij} = \begin{bmatrix} R_{z}^T R_{i}^T &  0 \\ 0 & 1 \end{bmatrix}$

And the following code shows the corresponding calculations in Python:

In [192]:
si = np.sin(v_i[2])
ci = np.cos(v_i[2])
dr_i = np.array([[-si, ci], [-ci, -si]]).T
dt_ij = np.array([v_j[:2] - v_i[:2]]).T

t_i = v2t(v_i)
t_j = v2t(v_j)
r_i = t_i[:2,:2]
r_z = t_z[:2,:2]

a_ij = np.vstack((np.hstack((-r_z.T @ r_i.T, (r_z.T @ dr_i.T) @ dt_ij)), 
                         [0, 0, -1]))
print(f'The shape of A_ij is {a_ij.shape}')

b_ij = np.vstack((np.hstack((r_z.T @ r_i.T, np.zeros((2,1)))),
                         [0, 0, 1]))
print(f'The shape of B_ij is {b_ij.shape}')

The shape of A_ij is (3, 3)
The shape of B_ij is (3, 3)


Having jacobians we can calulate blocks of the H and b matrices:

In [197]:
omega = np.identity(3) # it should be the information matrix associated with the edge
h_ii =  a_ij.T @ omega @ a_ij
h_ij =  a_ij.T @ omega @ b_ij
h_jj =  b_ij.T @ omega @ b_ij
b_i  = -a_ij.T @ omega @ e
b_j  = -b_ij.T @ omega @ e

And because we calculated only partials blocks for global H and b matrices we have to insert them into apropriate places:

In [212]:
# Create new H and b matrices in the begining of an optimization iteration

num_nodes = 6  # should be taken from a graph
num_params = 3  # tx, ty, theta
H = np.zeros((num_nodes * num_params, num_nodes * num_params)) 
b = np.zeros((num_nodes * num_params, 1)) 

def id2index(id):
  return slice((num_params*id), (num_params*(id+1)))

i_id = 0
j_id = 2
H[id2index(i_id), id2index(i_id)] += h_ii
H[id2index(i_id), id2index(j_id)] += h_ij
H[id2index(j_id), id2index(i_id)] += h_ij.T
H[id2index(j_id), id2index(j_id)] += h_jj
b[id2index(i_id)] += b_i
b[id2index(j_id)] += b_j